##Add Drive

In [1]:
from google.colab import drive
import os
drive.mount("/content/drive/")

mypath = "drive/My Drive/CS598DLHProject"
os.listdir(mypath)

ModuleNotFoundError: No module named 'google.colab'

In [2]:
!pip install google-colab

     |████████████████████████████████| 72 kB 573 kB/s eta 0:00:01
     |████████████████████████████████| 64 kB 791 kB/s eta 0:00:01
     |████████████████████████████████| 104 kB 1.0 MB/s eta 0:00:01
     |████████████████████████████████| 758 kB 570 kB/s eta 0:00:01
     |████████████████████████████████| 8.0 MB 1.4 MB/s eta 0:00:01
     |████████████████████████████████| 11.8 MB 484 kB/s eta 0:00:01
     |████████████████████████████████| 57 kB 909 kB/s eta 0:00:01
     |████████████████████████████████| 484 kB 1.0 MB/s eta 0:00:01
     |████████████████████████████████| 245 kB 350 kB/s eta 0:00:01
     |████████████████████████████████| 132 kB 1.3 MB/s eta 0:00:01
     |████████████████████████████████| 561 kB 1.4 MB/s eta 0:00:01
     |████████████████████████████████| 73 kB 777 kB/s eta 0:00:01
     |████████████████████████████████| 133 kB 733 kB/s eta 0:00:01
     |████████████████████████████████| 118 kB 1.3 MB/s eta 0:00:01
     |████████████████████████████████| 58 kB 1.8 M

##Pull data from Drive

In [1]:
import gzip
import pickle as pkl
import gc


DATA_DIR =  "../data/"

X_NOTES_INDEXED_FILE = f'{DATA_DIR}X_NOTES_INDEXED.gz'
Y_ICD9_FILE = f'{DATA_DIR}Y_ICD9.gz'
Y_ICD9_ROLLED_FILE = f'{DATA_DIR}Y_ICD9_ROLLED.gz'


SPLIT_CONSTANT = 100000
# X dataset
# if os.path.exists(X_NOTES_INDEXED_FILE):
print ('reading from saved file X_NOTES_INDEXED_FILE: ', X_NOTES_INDEXED_FILE)
with gzip.open(X_NOTES_INDEXED_FILE, "rb") as f:
    X = pkl.load(f)
print('X.type: ', type(X))
print('X.shape: ', X.shape)

X_small = X[:SPLIT_CONSTANT]
del X
gc.collect()
print('X_small.type: ', type(X_small))
print('X_small.shape: ', X_small.shape)

# Y_ICD9 dataset
# if os.path.exists(Y_ICD9_FILE):
# print ('reading from saved file Y_ICD9_FILE: ', Y_ICD9_FILE)
# with gzip.open(Y_ICD9_FILE, "rb") as f:
#     Y_ICD9 = pkl.load(f)
# print('Y_ICD9.type: ',type(Y_ICD9))
# print('Y_ICD9.shape: ', Y_ICD9.shape)

# Y_ICD9_ROLLED dataset
# if os.path.exists(Y_ICD9_ROLLED_FILE):
print ('reading from saved file Y_ICD9_ROLLED_FILE: ', Y_ICD9_ROLLED_FILE)
with gzip.open(Y_ICD9_ROLLED_FILE, "rb") as f:
    Y_ICD9_ROLLED = pkl.load(f)
print('Y_ICD9_ROLLED.type: ',type(Y_ICD9_ROLLED))
print('Y_ICD9_ROLLED.shape: ', Y_ICD9_ROLLED.shape)

Y_small = Y_ICD9_ROLLED[:SPLIT_CONSTANT]

del Y_ICD9_ROLLED
gc.collect()
print('Y_small.type: ',type(Y_small))
print('Y_small.shape: ', Y_small.shape)


reading from saved file X_NOTES_INDEXED_FILE:  ../data/X_NOTES_INDEXED.gz
X.type:  <class 'numpy.ndarray'>
X.shape:  (399631, 2200)
X_small.type:  <class 'numpy.ndarray'>
X_small.shape:  (100000, 2200)
reading from saved file Y_ICD9_ROLLED_FILE:  ../data/Y_ICD9_ROLLED.gz
Y_ICD9_ROLLED.type:  <class 'numpy.ndarray'>
Y_ICD9_ROLLED.shape:  (399631, 781)
Y_small.type:  <class 'numpy.ndarray'>
Y_small.shape:  (100000, 781)


In [2]:
import numpy as np
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing import sequence

In [3]:
from tensorflow.keras import Model
from tensorflow.keras.layers import Embedding, GlobalAveragePooling1D, Dense, Conv1D, GlobalMaxPool1D

class CNN(Model):

    def __init__(self,
                 maxlen,
                 max_features,
                 embedding_dims,
                 class_num=1,
                 last_activation='relu'):
        super(CNN, self).__init__()
        self.maxlen = maxlen
        self.max_features = max_features
        self.embedding_dims = embedding_dims
        self.class_num = class_num
        self.last_activation = last_activation
        self.embedding = Embedding(self.max_features, self.embedding_dims, input_length=self.maxlen)
        self.conv = Conv1D(250, 3, activation=last_activation)
        self.max_pooling = GlobalMaxPool1D()
        self.classifier = Dense(self.class_num, activation='sigmoid')

    def call(self, inputs):
        if len(inputs.get_shape()) != 2:
            raise ValueError('The rank of inputs of FastText must be 2, but now is %d' % len(inputs.get_shape()))
        if inputs.get_shape()[1] != self.maxlen:
            raise ValueError('The maxlen of inputs of FastText must be %d, but now is %d' % (self.maxlen, inputs.get_shape()[1]))
        embedding = self.embedding(inputs)
        conv = self.conv(embedding)
        x = self.max_pooling(conv)
        output = self.classifier(x)
        return output

In [4]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(X_small, Y_small, test_size=0.2, random_state=42)

print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

(80000, 2200) (80000, 781)
(20000, 2200) (20000, 781)


In [5]:
import keras

max_features = 54000
maxlen = 2200
batch_size = 32
embedding_dims = 50
epochs = 10
class_num = 781


print('Build model...')
model = CNN(maxlen, max_features, embedding_dims, class_num)

# model = CNN3Layer(maxlen, max_features, embedding_dims)

# model.compile('adam', 'binary_crossentropy', metrics=['accuracy'])

model.compile('adam', 'binary_crossentropy', metrics=[keras.metrics.Precision(), keras.metrics.Recall()])

print('Train...')
# early_stopping = EarlyStopping(monitor='val_accuracy', patience=5, mode='max')
# model.fit(x_train, y_train,
#           batch_size=batch_size,
#           epochs=epochs,
#           callbacks=[early_stopping],
#           validation_data=(x_test, y_test))

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          # callbacks=[early_stopping],
          validation_split=0.2)

Build model...


2022-04-26 17:00:09.092170: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Train...
Epoch 1/10
2000/2000 [==============================] - 738s 368ms/step - loss: 0.0648 - precision: 0.6181 - recall: 0.1366 - val_loss: 0.0545 - val_precision: 0.7816 - val_recall: 0.1649
Epoch 2/10
2000/2000 [==============================] - 803s 401ms/step - loss: 0.0521 - precision: 0.7601 - recall: 0.2232 - val_loss: 0.0498 - val_precision: 0.7524 - val_recall: 0.2685
Epoch 3/10
2000/2000 [==============================] - 729s 364ms/step - loss: 0.0475 - precision: 0.7792 - recall: 0.2963 - val_loss: 0.0464 - val_precision: 0.7830 - val_recall: 0.3122
Epoch 4/10
2000/2000 [==============================] - 18261s 9s/step - loss: 0.0440 - precision: 0.7973 - recall: 0.3528 - val_loss: 0.0441 - val_precision: 0.7964 - val_recall: 0.3522
Epoch 5/10
2000/2000 [==============================] - 592s 296ms/step - loss: 0.0412 - precision: 0.8111 - recall: 0.3971 - val_loss: 0.0426 - val_precision: 0.8014 - val_recall: 0.3788
Epoch 6/10
2000/2000 [==============================